In [104]:
import pandas as pd
import json 

## 1.0 Database RAW - Cleaning Step

In [ ]:
a = json.loads("ai_conversation_deep_full_export.json")

In [ ]:
a= pd.read_json("ai_conversation_deep_full_export.json")

In [ ]:
a["conversations"]

In [ ]:
def personal(row):
    try:
        x =row[0]["Personal"][0]["conversation-data"][0]["messages"]["values"]
    except:
        x = " "

    return x     

In [ ]:
a["personal_data"] = a["conversations"].apply(personal)

In [ ]:
a["real_state_data"] = a["conversations"].apply(personal)

In [ ]:
a["personal_data"] = a["conversations"].apply(personal)

In [ ]:
a["personal_data"] = a["conversations"].apply(personal)

In [ ]:
a.iloc[31]["personal_data"]

In [ ]:
import json

def extract_messages(data):
    cleaned_data = []
    try:
        for item in data:
            fields = item.get("mapValue", {}).get("fields", {})
            sender = fields.get("sender", {}).get("stringValue", "")
            message = fields.get("message", {}).get("stringValue", "")
            
            if sender and message:
                cleaned_data.append({sender: message})
    except:
        pass
    return cleaned_data

In [ ]:
a["personal_cleaned"] = a["personal_data"].apply(extract_messages)

In [ ]:
conversation = a["personal_cleaned"].iloc[1]

In [ ]:
a.to_csv("sample_data.csv")

## 2.0 Processing the cleaned Sheet

In [ ]:

# Separate AI and Human messages into lists
ai_messages = []
human_messages = []

# Iterate through the conversation and separate messages
for msg in conversation:
    if 'AI' in msg:
        ai_messages.append(msg['AI'])
        human_messages.append("")  # Placeholder for missing human responses
    elif 'human' in msg:
        human_messages[-1] = msg['human']  # Assign human response to last AI question

# Create DataFrame
dx = pd.DataFrame({'AI': ai_messages, 'Human': human_messages})


In [77]:
x = a["personal_cleaned"].drop_duplicates()

In [84]:
ai_messages = []
human_messages = []

In [ ]:
for conversation in x:
    if conversation == []:
        pass
    else:
        print(conversation)

        for msg in conversation:
            if 'AI' in msg:
                ai_messages.append(msg['AI'])
                human_messages.append("")  # Placeholder for missing human responses
            elif 'human' in msg:
                human_messages[-1] = msg['human']  

[{'AI': 'Hi! Would you like to fill your Personal data?'}, {'human': 'Yes'}, {'AI': 'Can you provide details about your prior marital status, as this may impact your current legal or financial obligations?'}, {'human': 'Married'}, {'AI': 'Have you ever been divorced, as this information may impact your legal or financial obligations?'}, {'human': 'Never'}, {'AI': 'Which branch of military service did you serve in, as this information is crucial for accessing veteran benefits and medical care?'}, {'human': 'Never Served'}, {'AI': 'What specific wishes do you have regarding your asset distribution in your estate planning?'}, {'human': 'Can you please provide more information?'}, {'AI': "For effective estate planning, it's essential to ensure that your assets are distributed according to your wishes. This involves creating a will or trust that clearly outlines how you would like your assets to be divided among your loved ones. \n\nCould you share more about your current estate planning ar

In [86]:
dx = pd.DataFrame({'AI': ai_messages, 'Human': human_messages})


In [88]:
dx.to_csv("personal_question.csv")

In [89]:
dx

,AI,Human
0,Hi! Would you like to fill your Personal data?,Yes
1,Can you provide details about your prior marit...,Married
2,"Have you ever been divorced, as this informati...",Never
3,Which branch of military service did you serve...,Never Served
4,What specific wishes do you have regarding you...,Can you please provide more information?
...,...,...
582,Hi! Would you like to fill your Personal details?,hello
583,Can you provide details about your previous ma...,hello
584,Hi! Would you like to fill your Personal data?,HI
585,What was your marital status prior to this mar...,I'M GOOD


In [90]:
from openai import OpenAI
import json
 
client = OpenAI()
 
def validate_response_new(question: str, answer: str) -> dict:
    """
    Validates whether a given response is valid based on predefined rules.
 
    Parameters:
        question (str): The question asked.
        answer (str): The human response.
 
    Returns:
        dict: A structured response (e.g., {"result": "yes", "location": "drawer", "reason": "Location provided"})
    """
 
    prompt = f"""
    You are in a Q&A session, validating whether an answer satisfies a question to gather information.
 
    **Rules:**
    1. If the answer contains a location (e.g., "in my drawer", "at home", "in New York"), return:
       {{
          "result": "yes",
          "location": "<EXTRACTED LOCATION>",
       }}
    2. If the answer includes "Yes" or similar, return:
       {{
          "result": "yes",
       }}
    3. If the answer includes "Maybe" or similar, return:
       {{
          "result": "loop",
       }}
    4. If the answer includes "No", "Skip", or similar, return:
       {{
          "result": "skip",
          
       }}
    5. If the answer is a question, return:
       {{
          "result": "question",
       }}
 
    **Strictly return only valid JSON. Do NOT include explanations or extra text.**
   
    **Question:** {question}  
    **Answer:** {answer}
    """
 
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[{"role": "system", "content": prompt}],
        response_format={"type": "json_object"} 
    )
 
    response_data = json.loads(completion.choices[0].message.content)  

    try:
        prompt_result = response_data["result"]

        if prompt_result.lower()== "yes":
            prompt_result = "answered"
    except:
        prompt_result = "skip" 
 
    return prompt_result 

In [91]:
dx.columns

Index(['AI', 'Human'], dtype='object')

In [ ]:
dx["prompt_result"]=

In [92]:
dx["AI_Validated"] = dx.apply(lambda row: validate_response_new(row["AI"], row["Human"]), axis=1)


In [93]:
dx.to_csv("dx.csv")

In [97]:
dx2 = dx.drop_duplicates(subset=["AI"])

In [99]:
dx2.to_csv("final.csv")

In [102]:
validate_response_new("What specific long-term care instructions do you have in mind, such as dietary restrictions or medication management for someone with a condition like diabetes?","i have a long-term care from morgan stanley")

'skip'

## 3.0 New prompts march 19

In [108]:
from openai import OpenAI
import json
 
client = OpenAI()
 
def validate_response_new2(question: str, answer: str) -> dict:
    """
    Validates whether a given response is valid based on predefined rules.
 
    Parameters:
        question (str): The question asked.
        answer (str): The human response.
 
    Returns:
        dict: A structured response (e.g., {"result": "yes", "location": "drawer", "reason": "Location provided"})
    """
 
    prompt = f"""
    You are in a Q&A session, validating whether an answer satisfies a question to gather information.
 
    **Rules:**
    1. If the answer contains a location (e.g., "in my drawer", "at home", "in New York"), return:
       {{
          "result": "yes",
          "location": "<EXTRACTED LOCATION>",
       }}
    2. If the answer includes "Yes" or similar, return:
       {{
          "result": "yes",
       }}
    3. If the answer includes "Maybe" or similar, return:
       {{
          "result": "loop",
       }}
    4. If the answer includes "No", "Skip", or similar, return:
       {{
          "result": "skip",
          
       }}
    5. If the answer is a question, return:
       {{
          "result": "question",
       }}
    
    6. If the answer contains both a response (e.g., "Yes, I have") **and** a question (e.g., "Can you explain more?"), or similar, return:
       {{
          "result": "question",
          "message": "The user partially answered but needs clarification. Explain again before proceeding."
       }} 
       
    **Strictly return only valid JSON. Do NOT include explanations or extra text.**
   
    **Question:** {question}  
    **Answer:** {answer}
    """
 
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[{"role": "system", "content": prompt}],
        response_format={"type": "json_object"} 
    )
 
    response_data = json.loads(completion.choices[0].message.content)  

    try:
        prompt_result = response_data["result"]

        if prompt_result.lower()== "yes":
            prompt_result = "answered"
    except:
        prompt_result = "skip" 
 
    return prompt_result 

In [111]:
df = pd.read_csv("Test Set Validation - final_sheet_prompt.csv")

In [136]:
df

,ai,human/ response from user,ground truth,ai_validated/test result,human validation,unnamed: 5,unnamed: 6,unnamed: 7,ai_validated_part2,final_result
0,Hi! Would you like to fill your Personal data?,yes,answered,answered,Correct Result,NaN,NaN,NaN,answered,True
1,Can you provide details about your prior marit...,married,answered,answered,Correct Result,NaN,NaN,NaN,answered,True
2,"Have you ever been divorced, as this informati...",never,skip,skip,Correct Result,NaN,NaN,NaN,skip,True
3,Which branch of military service did you serve...,never served,skip,skip,Correct Result,NaN,Gound Truth,The Values that should be output from bot (Cor...,skip,True
4,What specific wishes do you have regarding you...,can you please provide more information?,question,question,Correct Result,NaN,Ai Validated Results,Responses that are coming from AI prompts,question,True
...,...,...,...,...,...,...,...,...,...,...
281,What type of credit cards do you currently hav...,i have master credit card and i will manage to...,answered,answered,Correct Result,NaN,NaN,NaN,answered,True
282,What is the location of the physical card issu...,in city bank,answered,answered,Correct Result,NaN,NaN,NaN,answered,True
283,Can you provide details about your previous ma...,i am unmarried,answered,skip,Correct Result,NaN,NaN,NaN,skip,False
284,Can you provide details about your previous ma...,hello,loop,question,Incorrect Result,NaN,NaN,NaN,question,False


In [113]:
df["AI_Validated_part2"] = df.apply(lambda row: validate_response_new2(row["AI"], row['Human/ Response from User']), axis=1)


In [120]:
df.columns = df.columns.str.lower()

In [124]:
df["human/ response from user"] = df["human/ response from user"].str.lower()


In [126]:
df["ai_validated_part2"] = df["ai_validated_part2"].str.lower()

In [131]:
df["ground truth"] = df["ground truth"].str.lower()

In [132]:
df["final_result"] = df["ground truth"] == df["ai_validated_part2"]

In [134]:
df.to_csv("march_20.csv")

## 4.0 Analysing the wrong response

In [139]:
wrong_response = df[df["final_result"]== False]

In [140]:
wrong_response.shape

(36, 10)

In [143]:
wrong_response.to_csv("sdf.csv")

In [ ]:
from openai import OpenAI
import json
 
client = OpenAI()
 
def validate_response_new2(question: str, answer: str) -> dict:
    """
    Validates whether a given response is valid based on predefined rules.
 
    Parameters:
        question (str): The question asked.
        answer (str): The human response.
 
    Returns:
        dict: A structured response (e.g., {"result": "yes", "location": "drawer", "reason": "Location provided"})
    """
 
    prompt = f"""
    You are in a Q&A session, validating whether an answer satisfies a question to gather information.
 
    **Rules:**
    1. If the answer contains a location (e.g., "in my drawer", "at home", "in New York"), return:
       {{
          "result": "yes",
          "location": "<EXTRACTED LOCATION>",
       }}
    2. If the answer includes "Yes" or similar, return:
       {{
          "result": "yes",
       }}
    3. If the answer includes "Maybe" or similar, return:
       {{
          "result": "question",
       }}
    4. If the answer includes "No", "Skip", or similar, return:
       {{
          "result": "skip",
          
       }}
    5. If the answer is a question, return:
       {{
          "result": "question",
       }}
    
    6. If the answer contains both a response (e.g., "Yes, I have") **and** a question (e.g., "Can you explain more?"), or similar, return:
       {{
          "result": "question",
          "message": "The user partially answered but needs clarification. Explain again before proceeding."
       }} 
       
    **Strictly return only valid JSON. Do NOT include explanations or extra text.**
   
    **Question:** {question}  
    **Answer:** {answer}
    """
 
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[{"role": "system", "content": prompt}],
        response_format={"type": "json_object"} 
    )
 
    response_data = json.loads(completion.choices[0].message.content)  

    try:
        prompt_result = response_data["result"]

        if prompt_result.lower()== "yes":
            prompt_result = "answered"
    except:
        prompt_result = "skip" 
 
    return prompt_result 